# Classifying game genres based on game descriptions
The goal is to use short (170 character) description of games to classify them as 1 og 31 game genres. The idea is that it should be possible to get a broad overview of what kind og game it is by reading the description. Let's have a few example:

>a game of exploration and survival in an epic fantasy setting

>A physics based game with 100 levels of challenging fun

>Welcome to the next-generation of open-world gaming

>The long awaited second expansion to Blizzards critically acclaimed RTS

>A Pay2Win business model simulator(adventure) that exposes the tricks online game businessmen use to maximize monetizatio

This illustrages some of the benifits and challanges of the dataset. Let's have a look at the 31 genres:

1. Action
2. Indie
3. Strategy
4. Early Access
5. Free to Play
6. Massively Multiplayer
7. RPG
8. Adventure
9. Casual
10. Simulation
11. Racing
12. Sports
13. **Audio Production**
14. **Utilities**
15. **Video Production**
16. **Education**
17. **Design & Illustration**
18. **Web Publishing**
19. **Photo Editing**
20. **Software Training**
21. **Animation & Modeling**
24. Puzzle
25. Platformer
26. Survival
27. Shooter
28. Horror
29. Sandbox
30. **Music**
31. Fighting
32. Hidden Objects
33. **Accounting**

Quite a lot of these (11 ~ 1/3) are not actually games. That is because Steam sell application that are not games.

To do this we will be using the sklearn python package. It is an extremely useful machine learning package that will allow us to build a powerfull classifier in just a couple of lines. The disadvantage of this package is that it makes it possible to build, very quickly, something that you don't understand. At best it is a problem solver, at worst it is a black box generator.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

from sklearn.base import clone

from sklearn.pipeline import Pipeline
from sklearn import metrics

from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

## Getting and loading the data
The raw data are psql dumps from the homepage www.arcatetrail.com. The site is not longer active. 

The raw data is loaded using custom loading functions. 

In [2]:
import src.ReccomenderSystem as RC

titles = RC.reading_titles('data/game_titles.csv')
genres, num_games, num_attributes = RC.reading_votes('data/game_genres.csv')
id, taglines = RC.reading_taglines('data/game_tagline.csv')

For example, one of the taglines is the following:

In [3]:
print(taglines[1])

a game of exploration and survival in an epic fantasy setting



## Restructuring the data
Before we can use scikit-learn to build a classifier, we must make sure that the data is structured in a way scikit-learn will accept.

The classifier will take a string as imput and produce a numerical genre index. Below I make a list of all the game tags and a corresponding vector contining all the genre indices.

In [4]:
y_mat = np.zeros((len(id),32))
X_mat = []
X_titles = []
kk = 0;
for ii in range(0,len(id)):
    tagline_index = np.where(genres[:,0]==int(id[ii]))
    if len(tagline_index[0])>0:
        X_mat.append(taglines[ii])
        for jj in range(0,len(tagline_index[0])):
            genre = int(genres[tagline_index[0][jj],1])
            y_mat[kk,genre-1] = 1
    else:
        X_mat.append(taglines[ii])
    kk = kk+1

## Exploring the data
Before I build the classifier, I it is usefull to explore the data to get a realistic expection about how successful the classifier can be. Below I look at how the games are distributed across the different genres.

In [5]:
aggregate = sum(y_mat)
plt.bar(range(len(aggregate)),aggregate)
plt.show()

Many of the genres are empty. That is, there are no games with that genre. There are also quite a few of the genres that have less than 50 games in them. It will be difficult to build a good classifier for there.

## Splitting into train and test sets

In [6]:
train_size = int(round(len(y_mat)*0.8))
genre_to_train = 0
y_train = y_mat[:train_size, genre_to_train]
X_train = X_mat[:train_size]
X_test = X_mat[train_size:];
y_test = y_mat[train_size:, genre_to_train];

## Doing the pre-processing

In [7]:
X_train[0]

'A science fiction action role-playing third person shooter video game developed by BioWare and published by Electronic Arts.\n'

### Going from strings to vectors (Vectorizer)

In [8]:
tfid_vectorizer = TfidfVectorizer(X_train,
                stop_words='english',analyzer='word')

The stop words the model ignors can be inspected:

In [9]:
tfid_vectorizer.get_stop_words();

Likewise, the strings used to build the analyzer can be inspected togeter with the parameters of the model

In [10]:
tfid_vectorizer.get_params();

So lets see how it works by running it over an example and doing the inverse transform to get the words it used back

In [11]:
X_vect = tfid_vectorizer.fit_transform([X_test[0]])

In [12]:
X_inverse = tfid_vectorizer.inverse_transform(X_vect)

So the test example gets the following transform:

In [13]:
print('before: ' + X_test[0])
print('After: ' + str(X_inverse[0][:]))

before: A fast and skill based party game where all of the focus lies in fun and engaging gameplay

After: ['fast' 'skill' 'based' 'party' 'game' 'focus' 'lies' 'fun' 'engaging'
 'gameplay']


## Lemmatizing

In [14]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [15]:
tfid_vectorizer = TfidfVectorizer(X_train,
                    tokenizer=LemmaTokenizer(), 
                    stop_words='english',analyzer='word')


In [16]:
X_vect = tfid_vectorizer.fit_transform([X_test[0]])
X_inverse = tfid_vectorizer.inverse_transform(X_vect)
print('before: ' + X_test[0])
print('After: ' + str(X_inverse[0][:]))

before: A fast and skill based party game where all of the focus lies in fun and engaging gameplay

After: ['fast' 'skill' 'based' 'party' 'game' 'focus' 'lie' 'fun' 'engaging'
 'gameplay']


### Normalizing with term-frequency-inverse-document-frequency

In [17]:
tfidf_normalizer = TfidfTransformer(norm='l2',use_idf=True,smooth_idf=True)

In [18]:
X_vect_norm = tfidf_normalizer.fit_transform(X_vect)

In [19]:
print(X_vect_norm)

  (0, 6)	0.316227766017
  (0, 1)	0.316227766017
  (0, 4)	0.316227766017
  (0, 7)	0.316227766017
  (0, 3)	0.316227766017
  (0, 5)	0.316227766017
  (0, 8)	0.316227766017
  (0, 0)	0.316227766017
  (0, 9)	0.316227766017
  (0, 2)	0.316227766017


## Setting up the pipeline

In [20]:
pipeline = Pipeline([
('vect', tfid_vectorizer),
('tfidf', tfidf_normalizer),
('clf', MultinomialNB())
])
clf = pipeline.fit(X_train, y_train)

## Trying out a few made-up examples

In [21]:
genre_names = ['Action', 'Indie', 'Strategy', 'Early Access', 'Free to Play', 'Massively Multiplayer', 'RPG',
               'Adventure', 'Casual', 'Simulation', 'Racing', 'Sports', 'Audio Production', 'Utilities', 
               'Video Production', 'Education', 'Design & Illustration', 'Web Publishing', 'Photo Editing',
               'Software Training', 'Animation & Modeling', 'Puzzle', 'Platformer', 'Survival', 'Shooter', 
               'Horror', 'Sandbox', 'Music', 'Fighting', 'Hidden Objects', 'Accounting']

In [22]:
model_collection = []
for genre in range(0,31):
    clf_clone = clone(pipeline)
    clf_clone.fit(X_mat, y_mat[:, genre])
    model_collection.append(clf_clone)

/Users/hartmann/Documents/projects/arcade_trail/env/lib/python3.6/site-packages/sklearn/naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/hartmann/Documents/projects/arcade_trail/env/lib/python3.6/site-packages/sklearn/naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/hartmann/Documents/projects/arcade_trail/env/lib/python3.6/site-packages/sklearn/naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
/Users/hartmann/Documents/projects/arcade_trail/env/lib/python3.6/site-packages/sklearn/naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


In [32]:
example = ['Shoot your way through the jungle to survive']

In [33]:
pred_list = []
for idx, clf in enumerate(model_collection):
    pred = clf.predict_proba(example)
    pred_list.append(pred[:,1])
    #print(pred[:,1], genre_names[idx])

In [34]:
sort_idx = sorted(range(len(pred_list)), key=lambda k: pred_list[k], reverse=True)
my_list = [(pred_list[idx], genre_names[idx]) for idx in sort_idx]
my_list

[(array([ 0.70640605]), 'Indie'),
 (array([ 0.69288501]), 'Action'),
 (array([ 0.19967321]), 'Adventure'),
 (array([ 0.09826113]), 'Casual'),
 (array([ 0.06818273]), 'RPG'),
 (array([ 0.05643554]), 'Strategy'),
 (array([ 0.05127073]), 'Early Access'),
 (array([ 0.04673651]), 'Simulation'),
 (array([ 0.01127945]), 'Free to Play'),
 (array([ 0.01115405]), 'Sports'),
 (array([ 0.00724097]), 'Massively Multiplayer'),
 (array([ 0.00634974]), 'Racing'),
 (array([ 0.0051457]), 'Sandbox'),
 (array([ 0.00289246]), 'Survival'),
 (array([ 0.00288142]), 'Shooter'),
 (array([ 0.00077525]), 'Design & Illustration'),
 (array([ 0.0007749]), 'Hidden Objects'),
 (array([ 0.00062086]), 'Web Publishing'),
 (array([ 0.00061943]), 'Horror'),
 (array([ 0.00046605]), 'Animation & Modeling'),
 (array([ 0.00046545]), 'Utilities'),
 (array([ 0.00046532]), 'Music'),
 (array([ 0.00031146]), 'Accounting'),
 (array([ 0.00031129]), 'Video Production'),
 (array([ 0.00031095]), 'Audio Production'),
 (array([ 0.00031083

It seems that the model is giving the same probability to every genre. This is a bit strange. It is probably work looking into what words activate the different genres. It may be that is it standard words such as "The" and "and". Looking into the tokenizer and stemming may help. It should also be possible to get rid of unusefull words.

## Cool applications of the classifier described above
- classify.me: A website that classifies a game given a tagline.
- tagline generator: One could run the algorithm in reverse. Given a genre, generate a tagline. I'm not entirely sure how one would do this. The idea is to get some of the top keywords from a genre and then use those to generate a short description. I'm sure other people have worked with generating sentences string from keywords. See eg. [Narrative Science](#https://www.narrativescience.com/) 